## Importações e Funções de Apoio
---

In [53]:
import pandas as pd
import os
import re
import warnings
import json
import unicodedata
from io import StringIO
import string

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

In [54]:
def ajustes(x):
    x = str(x).strip()
    return 0 if x == "-" or x == "R$ -" else x

def strip(x):
    # Converter para string e remover espaços iniciais e finais
    x = str(x).strip()

    # Remover acentos
    x = ''.join(
        c for c in unicodedata.normalize('NFKD', x)
        if not unicodedata.combining(c)
    )

    # Remover pontuações
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = str(x).upper()

    return x

dia_da_semana = {
    "Sunday": "Domingo",
    "Monday": "Segunda-feira",
    "Tuesday": "Terça-feira",
    "Wednesday": "Quarta-feira",
    "Thursday": "Quinta-feira",
    "Friday": "Sexta-feira",
    "Saturday": "Sábado"
}

In [55]:
with open(r"C:\Users\divtec-dados\DataspellProjects\Notebooks\Ceasa\Dados\JSON\lista_nomes_produtos_ceasa_online.json", "r", encoding="utf-8") as file:
	dicionario_nomes_produtos = json.load(file)
	for item in dicionario_nomes_produtos:
		dicionario_nomes_produtos[item] = dicionario_nomes_produtos[item]

In [56]:
df_pesos = pd.read_csv(r'C:\Users\divtec-dados\DataspellProjects\Notebooks\Ceasa\Dados\pesos_produtos_ceasa_online.csv', sep=";", decimal=",")
df_pesos['PESO'] = df_pesos['PESO'].astype('float')
df_pesos['CODIGO'] = df_pesos['CODIGO'].astype('string')

## Carregando e ajustes dos dados
---

In [57]:
caminho = r"C:\Users\divtec-dados\DataspellProjects\Notebooks\Ceasa\Dados\Cotações\Histórico de Cotações Ceasa.csv"
dados_brutos = pd.read_csv(caminho, sep=";", decimal=",")
dados_brutos['DATA'] = pd.to_datetime(dados_brutos['DATA'], format='%d/%m/%Y', dayfirst=True)
df = dados_brutos.copy(deep=True)
df[['MÁXIMO','MÍNIMO','M COMUM']] = df[['MÁXIMO','MÍNIMO','M COMUM']].astype(float)
df['PRODUTO'] = df['PRODUTO'].map(dicionario_nomes_produtos)
df

,DATA,PRODUTO,UNIDADE,MÁXIMO,M COMUM,MÍNIMO
0,2020-01-02,Abacate Comum,KG,5.00,5.00,4.00
1,2020-01-02,Abacaxi Caiena (Hawai),UND,4.00,3.80,3.00
2,2020-01-02,Abacaxi Perola,UND,3.50,3.50,2.50
3,2020-01-02,Abacaxi Terra de Areia,UND,3.00,2.50,2.00
4,2020-01-02,Abobora,KG,1.75,1.50,1.20
...,...,...,...,...,...,...
126164,2024-12-30,Verdes Ruscus,MOL,30.00,30.00,30.00
126165,2024-12-30,Violeta,UND,6.90,6.90,5.00
126166,2024-12-30,Violeta Mini,CX,NaN,NaN,NaN
126167,2024-12-30,Yacon,KG,19.00,15.00,10.56


## Manipulação dos Dados
---

## Verificação das Datas
---

In [58]:
df['DIA_DA_SEMANA'] = df['DATA'].dt.day_name()
df['DIA_DA_SEMANA'] = df['DIA_DA_SEMANA'].map(dia_da_semana)
df

,DATA,PRODUTO,UNIDADE,MÁXIMO,M COMUM,MÍNIMO,DIA_DA_SEMANA
0,2020-01-02,Abacate Comum,KG,5.00,5.00,4.00,Quinta-feira
1,2020-01-02,Abacaxi Caiena (Hawai),UND,4.00,3.80,3.00,Quinta-feira
2,2020-01-02,Abacaxi Perola,UND,3.50,3.50,2.50,Quinta-feira
3,2020-01-02,Abacaxi Terra de Areia,UND,3.00,2.50,2.00,Quinta-feira
4,2020-01-02,Abobora,KG,1.75,1.50,1.20,Quinta-feira
...,...,...,...,...,...,...,...
126164,2024-12-30,Verdes Ruscus,MOL,30.00,30.00,30.00,Segunda-feira
126165,2024-12-30,Violeta,UND,6.90,6.90,5.00,Segunda-feira
126166,2024-12-30,Violeta Mini,CX,NaN,NaN,NaN,Segunda-feira
126167,2024-12-30,Yacon,KG,19.00,15.00,10.56,Segunda-feira


## Padronização dos Dados para exportar ao COL

In [59]:
df_export = pd.merge(df, df_pesos, how='left', on=['PRODUTO','UNIDADE'])

In [60]:
df_export

,DATA,PRODUTO,UNIDADE,MÁXIMO,M COMUM,MÍNIMO,DIA_DA_SEMANA,CODIGO,PESO
0,2020-01-02,Abacate Comum,KG,5.00,5.00,4.00,Quinta-feira,218,1.00
1,2020-01-02,Abacaxi Caiena (Hawai),UND,4.00,3.80,3.00,Quinta-feira,2,1.80
2,2020-01-02,Abacaxi Perola,UND,3.50,3.50,2.50,Quinta-feira,3,1.30
3,2020-01-02,Abacaxi Terra de Areia,UND,3.00,2.50,2.00,Quinta-feira,4,0.60
4,2020-01-02,Abobora,KG,1.75,1.50,1.20,Quinta-feira,5,1.00
...,...,...,...,...,...,...,...,...,...
126164,2024-12-30,Verdes Ruscus,MOL,30.00,30.00,30.00,Segunda-feira,213,0.30
126165,2024-12-30,Violeta,UND,6.90,6.90,5.00,Segunda-feira,215,0.11
126166,2024-12-30,Violeta Mini,CX,NaN,NaN,NaN,Segunda-feira,250,1.50
126167,2024-12-30,Yacon,KG,19.00,15.00,10.56,Segunda-feira,216,1.00


In [61]:
df_export.loc[df_export['CODIGO'].isna()]

,DATA,PRODUTO,UNIDADE,MÁXIMO,M COMUM,MÍNIMO,DIA_DA_SEMANA,CODIGO,PESO
19442,2020-11-24,Violeta,KG,0.00,0.00,0.00,Terça-feira,<NA>,NaN
19601,2020-11-26,Violeta,KG,0.00,0.00,0.00,Quinta-feira,<NA>,NaN
125500,2024-12-26,Tâmara Importado(a),KG,30.00,30.00,30.00,Quinta-feira,<NA>,NaN
125501,2024-12-26,Tâmara Importado(a),KG,NaN,NaN,NaN,Quinta-feira,<NA>,NaN
125502,2024-12-26,Tamarindo Importado(a),KG,36.67,36.67,36.67,Quinta-feira,<NA>,NaN
125746,2024-12-27,Tâmara Importado(a),KG,30.00,30.00,30.00,Sexta-feira,<NA>,NaN
125747,2024-12-27,Tâmara Importado(a),KG,NaN,NaN,NaN,Sexta-feira,<NA>,NaN
125748,2024-12-27,Tamarindo Importado(a),KG,36.67,36.67,36.67,Sexta-feira,<NA>,NaN
125992,2024-12-29,Tâmara Importado(a),KG,30.00,30.00,30.00,Domingo,<NA>,NaN
125993,2024-12-29,Tâmara Importado(a),KG,NaN,NaN,NaN,Domingo,<NA>,NaN
